In [1]:
import networkx as nx
from pyvis.network import Network
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('raw_data/cartel_shareholders/ABB.xlsx')

# drop last 3 rows
df = df[:-3]

df_cleaned = df.dropna(how='all').dropna(axis=1, how='all')

# Get relevant columns with % O/S in the name
relevant_columns_percent = df_cleaned.columns[df_cleaned.columns.str.contains('% O/S')]
relevant_columns_filing = df_cleaned.columns[df_cleaned.columns.str.contains('Filing Type')]

df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

# Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
col_list = []

for col in df_cleaned.columns:
    if '% O/S' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'perc_os_{month_year}')
    elif 'Filing Type' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'filing_type_{month_year}')
    else:
        col_list.append(str.lower(str.replace(col, ' ', '_')))

df_cleaned.columns = col_list

df_cleaned.head(10)

,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2009,filing_type_31-Dec-2008,filing_type_31-Dec-2007,filing_type_31-Dec-2006,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000
0,Ålandsbanken Asset Management Ab,Investment Advisor,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wilmington Trust Investment Management LLC,Bank and Trust,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"William Blair & Company, L.L.C.",Investment Advisor/Hedge Fund,NaN,NaN,NaN,NaN,0.00,0.57,NaN,NaN,...,NaN,NaN,13F,13F,NaN,NaN,NaN,NaN,NaN,NaN
3,Wells Capital Management Inc.,Investment Advisor/Hedge Fund,0,0.07,0.07,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Warburg Invest Kapitalanlagegesellschaft mbH,Investment Advisor,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,...,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Vontobel Asset Management, Inc.",Investment Advisor/Hedge Fund,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Veritas Investment Trust GmbH,Investment Advisor,NaN,NaN,NaN,0.17,0.17,NaN,NaN,NaN,...,NaN,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Vanguard Group, Inc.",Investment Advisor,0.01,0.00,0.25,0.20,0.17,NaN,NaN,NaN,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,NaN,NaN,NaN,0.00,0.01,NaN,NaN,NaN,...,NaN,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Udwadia (D E),Individual Investor,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,Other Substantial/Declarable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Get all Files in raw_data/cartel_shareholders folder
import os
files = os.listdir("raw_data/cartel_shareholders")
files = [f for f in files if f.endswith(".xlsx")]

print(len(files))

# Create a list of all the dataframes
dfs = []
for f in files:
    df = pd.read_excel("raw_data/cartel_shareholders/" + f)
    
        # drop last 3 rows
    df = df[:-3]

    # Drop only empty rows
    df_cleaned = df.dropna(how='all', axis=0)

    # Get relevant columns with % O/S in the name
    relevant_columns_percent = df_cleaned.columns[df_cleaned.columns.str.contains('% O/S')]
    relevant_columns_filing = df_cleaned.columns[df_cleaned.columns.str.contains('Filing Type')]

    df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

    # Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
    col_list = []

    for col in df_cleaned.columns:
        if '% O/S' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'perc_os_{month_year}')
        elif 'Filing Type' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'filing_type_{month_year}')
        else:
            col_list.append(str.lower(str.replace(col, ' ', '_')))

    df_cleaned.columns = col_list

    df_cleaned["cartel_name"] = f.split(".")[0]
    print(f.split(".")[0])
    dfs.append(df_cleaned)

dfs[10].head(10)

61
Aalberts
ABB
Akzo Nobel
Alstom SA
Areva
Asahi
AU Optronics
BAM
Bayern
Boliden
British Airways
Cathay Pacific
Chemtura
Chimei
chiquita
Chungwa
Commerzbank
del monte
Dow
Elpida
ENI
EON
Exxon Mobil
Fuji Electric
Fujifilm
GDF suez
Hannstar Display
Henkel
hitachi ltd
Hitachi Maxell
ICI
IMI PLC
Infineon
LG Display
Micron
Mitsubishi
Mueller Industries
Nanya Tech
NEC
Nippon electric glass
Panasonic
Pilkington
procter gamble
Qantas
rautaruukki
Repsol YPF
samsung
SAS AB
Siemens
Singapore Airlines
SKW Stahl
Sony
Toshiba
Total
Unilever NV
Unilever PLC
Unipetrol
United technologies corp
Uralita
Whirlpool
Zeon


,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000,filing_type_31-Dec-1999,filing_type_31-Dec-1998,filing_type_31-Dec-1997,cartel_name
0,Étoile Gestion,Investment Advisor,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,British Airways
1,eQ Asset Management Oy,Investment Advisor,NaN,NaN,0.00,0.00,0.00,0.00,0.00,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
2,bfw fondsleitung AG,Investment Advisor,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
3,Zürcher Kantonalbank (Asset Management),Bank and Trust,NaN,0.02,0.06,0.07,0.02,0.06,0.07,0.04,...,UK Registers,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
4,Zurich Life Assurance plc,Insurance Company,NaN,0.09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
5,Wright Investors' Service Inc.,Investment Advisor,NaN,NaN,0.00,0.01,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
6,"World Asset Management, Inc.",Investment Advisor,NaN,0.00,0.00,0.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
7,Winterthur Investment Management Corporation,Insurance Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,British Airways
8,Wilmington Trust Investment Management LLC,Bank and Trust,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.02,...,UK Registers,UK Registers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
9,Williams de Broë Private Investment Management...,Investment Advisor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,...,NaN,UK Registers,UK Registers,UK Registers,NaN,NaN,NaN,NaN,NaN,British Airways


In [4]:
print(f"Number of dataframes: {len(dfs)}")

# Merge all dataframes into one
df_merged = pd.concat(dfs)

investor_names = df_merged['investor_name'].unique()
cartel_names = df_merged['cartel_name'].unique()

# print amount of uniques
print(f"Number of unique investors: {len(investor_names)}")
print(f"Number of unique cartels: {len(cartel_names)}")

# order investors by number of companies they are in invested in
investor_counts = df_merged['investor_name'].value_counts()

investor_counts = investor_counts.sort_values(ascending=False)

print(investor_counts)



Number of dataframes: 61
Number of unique investors: 8246
Number of unique cartels: 61
investor_name
Universal-Investment-Gesellschaft mbH                58
Dimensional Fund Advisors, LP                        56
TIAA-CREF                                            56
State Street Global Advisors (France) S.A.           56
Vanguard Group, Inc.                                 56
                                                     ..
Geisler (James E)                                     1
Garnier (Jean-Pierre)                                 1
Faraci (John V)                                       1
Euroamérica Administradora General de Fondos S.A.     1
David (George A L)                                    1
Name: count, Length: 8246, dtype: int64


In [5]:
for df in dfs:
    # Get all columns with perc_os in the name
    perc_os_columns = df.columns[df.columns.str.contains('perc_os')]

    # Get the name of the cartel e.g. ABB
    name = df["cartel_name"][0]

    # Print the name of the cartel
    print(name)

    # Create a graph for each column with perc_os in the name
    for column in perc_os_columns:
        G = nx.Graph()

        for _, row in df.iterrows():
            investor = row["investor_name"]
            firm = row["cartel_name"]
            weight = row[column]

            # Check if weight is not null
            if not pd.isna(weight):  
                G.add_node(firm, type="Cartel")
                G.add_node(investor, type="Investor")
                G.add_edge(investor, firm, weight=weight)
        
        # Save the graph as a graphml file
        nx.write_graphml(G, f"./transformed_data/shareholder_networks/shareholder_network_{name}_{str.replace(column, 'perc_os_', '')}.graphml", named_key_ids=True, infer_numeric_types=True)

Aalberts
ABB
Akzo Nobel
Alstom SA
Areva
Asahi
AU Optronics
BAM
Bayern
Boliden
British Airways
Cathay Pacific
Chemtura
Chimei
chiquita
Chungwa
Commerzbank
del monte
Dow
Elpida
ENI
EON
Exxon Mobil
Fuji Electric
Fujifilm
GDF suez
Hannstar Display
Henkel
hitachi ltd
Hitachi Maxell
ICI
IMI PLC
Infineon
LG Display
Micron
Mitsubishi
Mueller Industries
Nanya Tech
NEC
Nippon electric glass
Panasonic
Pilkington
procter gamble
Qantas
rautaruukki
Repsol YPF
samsung
SAS AB
Siemens
Singapore Airlines
SKW Stahl
Sony
Toshiba
Total
Unilever NV
Unilever PLC
Unipetrol
United technologies corp
Uralita
Whirlpool
Zeon


In [40]:
# List to store the final results in the desired format
top_investors_list = []

for df in dfs:
    # Get all columns that contain 'perc_os' in their name (i.e. ownership percentage per year)
    perc_os_columns = df.columns[df.columns.str.contains('perc_os')]

    # Convert these columns to numeric data type (in case they are strings)
    df[perc_os_columns] = df[perc_os_columns].apply(pd.to_numeric, errors='coerce')

    # Get the name of the cartel (e.g. "ABB")
    cartel_name = df["cartel_name"][0]

    for column in perc_os_columns:
        # Extract the year from the column name (e.g. 'perc_os_2020' → '2020')
        year = column[-4:]

        # Get the top 5 investors based on ownership percentage for that year
        top5 = df.nlargest(5, column)[["investor_name", column]].reset_index(drop=True)

        # Create a result dictionary for this cartel-year combination
        result = {
            "cartel_name": cartel_name,
            "year": year
        }

        # Add the top 5 investors and their ownership percentages
        for i in range(5):
            try:
                investor = top5.loc[i, "investor_name"]
                perc = top5.loc[i, column]

                 # If the percentage is NaN, set both to None
                if pd.isna(perc):
                    investor = None
                    perc = None

            except IndexError:
                # In case there are fewer than 5 investors
                investor = None
                perc = None

            result[f"{i+1}_investor_name"] = investor
            result[f"{i+1}_perc_os"] = perc

        # Append the result to the final list
        top_investors_list.append(result)

# Convert the result list into a DataFrame (optional)
top_investors_df = pd.DataFrame(top_investors_list)

# Sort the DataFrame by cartel_name (ascending) and year (ascending)
top_investors_df = top_investors_df.sort_values(by=["cartel_name", "year"], ascending=[True, True])

# Show the first 15 rows
top_investors_df.head(10)


,cartel_name,year,1_investor_name,1_perc_os,2_investor_name,2_perc_os,3_investor_name,3_perc_os,4_investor_name,4_perc_os,5_investor_name,5_perc_os
29,ABB,1997,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
28,ABB,1998,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
27,ABB,1999,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
26,ABB,2000,"Banca Cívica Gestión de Activos SGIIC, SA.",0.33,SG Asset Management (Singapore) Ltd.,0.20,None,NaN,None,NaN,None,NaN
25,ABB,2001,UTI Asset Management Co. Ltd.,2.18,Lloyd George Investment Management (Hong Kong)...,1.85,Aberdeen Asset Management (Asia) Ltd.,0.94,Morgan Stanley Investment Management (Singapore),0.47,Tata Asset Management Limited,0.45
24,ABB,2002,UTI Asset Management Co. Ltd.,5.16,Morgan Stanley Investment Management (India) P...,0.94,Aberdeen Asset Management (Asia) Ltd.,0.81,Lloyd George Investment Management (Hong Kong)...,0.66,J.P. Morgan Asset Management (Hong Kong) Ltd.,0.59
23,ABB,2003,UTI Asset Management Co. Ltd.,2.66,J.P. Morgan Asset Management (Hong Kong) Ltd.,1.26,ICICI Prudential Asset Management Co. Ltd.,1.24,Aberdeen Asset Management (Asia) Ltd.,1.23,HSBC Global Asset Management (India) Private L...,0.67
22,ABB,2004,Aberdeen Asset Management (Asia) Ltd.,1.91,J.P. Morgan Asset Management (Hong Kong) Ltd.,1.79,ARISAIG Partners (Asia) Pte. Ltd.,1.24,UTI Asset Management Co. Ltd.,0.80,Morgan Stanley Investment Management (Singapore),0.71
21,ABB,2005,Morgan Stanley Investment Management (India) P...,2.56,Aberdeen Asset Management (Asia) Ltd.,2.34,J.P. Morgan Asset Management (Hong Kong) Ltd.,2.27,ARISAIG Partners (Asia) Pte. Ltd.,0.98,Franklin Templeton Asset Management (India) Pv...,0.84
20,ABB,2006,J.P. Morgan Asset Management (Hong Kong) Ltd.,2.36,Morgan Stanley Investment Management (India) P...,2.31,Aberdeen Asset Management (Asia) Ltd.,2.28,"William Blair & Company, L.L.C.",0.57,HSBC Global Asset Management (Hong Kong) Limited,0.48
